# Módulo 2: Implementación de una técnica de aprendizaje máquina sin el uso de un framework


Jorge Eduardo de León Reyna - A00829759

## Carga de librerias

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import math

## Descripcion del dataset utilizado

- **Dataset utilizado:** Heart Failure Prediction
- **Descripción y contexto del problema a resolver:** Las enfermedades cardiovasculares son la principal causa de muerte en todo el mundo, cobrando cerca de 17.9 millones de vidas al año y representando el 31% de todas las muertes. La mayoría de estas muertes son por ataques cardíacos y accidentes cerebrovasculares, y muchas ocurren prematuramente en personas menores de 70 años. La insuficiencia cardíaca también es común debido a estas enfermedades. Un conjunto de datos con 11 características puede ayudar a predecir posibles problemas cardíacos. Las personas con enfermedades cardíacas o alto riesgo cardiovascular, debido a factores como hipertensión o diabetes, necesitan detección temprana y manejo, donde un modelo de aprendizaje automático puede ser muy útil.
- **Atributos o campos:**
  - Age: edad del paciente
      - años
  - Sex: sexo del paciente
      - M: Male, F: Female
  - ChestPainType: tipo de dolor de pecho
      - TA: Typical Angina, ATA: Atypical Angina, NAP: Non-Anginal Pain, ASY: Asymptomatic
  - RestingBP: presión arterial en reposo
      - mm Hg
  - Cholesterol: colesterol
      - mm/dl
  - FastingBS: azucar en la sangre en ayunas
      - 1: si FastingBS > 120 mg/dl, 0: otro
  - RestingECG: resultados de electrocardiograma en reposo
      - Normal: Normal, ST: Anomalía de la onda ST-T (inversiones de onda T y/o elevación o depresión del segmento ST > 0.05 mV), LVH: mostrando hipertrofia ventricular izquierda probable o definitiva según los criterios de Estes.
  - MaxHR: frecuencia cardíaca máxima alcanzada
      - Valor numero entre 60 y 202
  - ExerciseAngina: angina inducida por el ejercicio
      - Y: Yes, N: No
  - Oldpeak: Descenso antiguo = ST
      - Valor numerico
  - ST_Slope: La pendiente del segmento ST durante el ejercicio máximo
      - Up: upsloping, Flat: flat, Down: downsloping
  - HeartDisease: salida
      - 1: heart disease, 0: Normal
- **Enlace:** https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction?resource=download
- **Descripcion del problema:** En este dataset se busca hacer un modelo de clasificación, donde se tienen dos clases posibles para predecir: "heart disease(1)" y "normal(1)".








## Cargando Dataset

In [ ]:
df = pd.read_csv("/content/heart.csv")
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


#### Limpieza de datos inicial

In [ ]:
df = df.dropna()

df = df.drop_duplicates()

df = df.reset_index(drop=True)

## Matriz de correlación
Se busca encontrar las variables con mayor influencia sobre la variable objetivo ("charges") para utilizar las mismas en las pruebas del modelo.

In [ ]:
correlation_matrix = df.corr()
print(correlation_matrix)

                   Age  RestingBP  Cholesterol  FastingBS     MaxHR   Oldpeak  \
Age           1.000000   0.254399    -0.095282   0.198039 -0.382045  0.258612   
RestingBP     0.254399   1.000000     0.100893   0.070193 -0.112135  0.164803   
Cholesterol  -0.095282   0.100893     1.000000  -0.260974  0.235792  0.050148   
FastingBS     0.198039   0.070193    -0.260974   1.000000 -0.131438  0.052698   
MaxHR        -0.382045  -0.112135     0.235792  -0.131438  1.000000 -0.160691   
Oldpeak       0.258612   0.164803     0.050148   0.052698 -0.160691  1.000000   
HeartDisease  0.282039   0.107589    -0.232741   0.267291 -0.400421  0.403951   

              HeartDisease  
Age               0.282039  
RestingBP         0.107589  
Cholesterol      -0.232741  
FastingBS         0.267291  
MaxHR            -0.400421  
Oldpeak           0.403951  
HeartDisease      1.000000  


<ipython-input-7-becb50483c66>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = df.corr()


Se encuentra que las variables con mayor correlacion a la variable objetivo ("charges") son: **"Oldpeak", "Age" y "FastingBS"** por lo que seran las seleccionadas para realizar pruebas con el modelo de regresion logistica.

In [ ]:
len(df)

918

## Dividiendo dataset en set de entrenamiento y set de pruebas

In [ ]:
train, test = train_test_split(df, test_size=0.3, random_state=42)

## Definiendo hiperparametros iniciales

In [ ]:
# Crear lista con los hiper-parámetros iniciales (thetas)
theta = [0.05, 0.01]
# Cargar el valor del learning rate (alpha)
alpha = 0.01

## Definiendo funcion de hipotesis y parametros para la misma

In [ ]:
# Crear función lambda para la función de hipótesis
h = lambda x, theta: 1 / (1 + math.exp(-1 * (theta[0] + theta[1] * x)))

# Calcular el total de muestras a partir de los datos (n)
n = len(train)

# Definir total de iteraciones a ejecutar
iteraciones = 1000

## Iteraciones del modelo
En cada iteración debemos calcular el valor de la derivada de la función de costo, que se obtiene a partir de los datos estimados, $h_\theta(x_i)$, y reales, $y_i$ (recuerde que los estimados se obtienen con la función de hipótesis de la regresión logística):

$\frac{∂J_{θ}}{∂θ_0} = \frac{1}{n}\sum_{i=1}^{n}(h_θ(x_i)-y_i)$

$\frac{∂J_{θ}}{∂θ_1} = \frac{1}{n}\sum_{i=1}^{n}(h_θ(x_i)-y_i)x_i$

Además, debemos actualizar los valores de los hiper-parámetros, recordando que $\theta_j=\theta_j-\alpha \frac{∂J_{θ}}{∂θ_j}$:

## Definicion de funcion para el modelo de regresion logistica

In [ ]:
# Seleccionar la muestra a utilizar (Attendance o Homework)
def gradiente_descendiente(x_vector, y_vector, iteraciones, theta, alpha):
  #reiniciamos valores de theta
  theta_aux = theta
  alpha = 0.01

  # Implementar ciclo para iteraciones
  for i in range(0, iteraciones):
    # Crear acumuladores
    delta1_aux = 0
    delta2_aux = 0

    # Calcular delta para theta0 y para cada muestra
    for j in range(0, n):
      current_cost_function_value = h(x_vector.iloc[j], theta_aux)

      # Calcular delta para theta1 y para cada muestra
      delta1_aux += current_cost_function_value - y_vector.iloc[j]

      # Calcular delta para theta2 y para cada muestra
      delta2_aux += (current_cost_function_value - y_vector.iloc[j]) * x_vector.iloc[j]

    # Calcular sumatorias y promedio
    theta_aux[0] -= (alpha * (delta1_aux / n))
    theta_aux[1] -= (alpha * (delta2_aux / n))

  # Imprimir theta actualizado
  print("theta final 1 => ", theta_aux[0])
  print("theta final 2 => ", theta_aux[1])

  return theta_aux

### Modelo 1 (prueba con variable "OldPeak")

In [ ]:
theta_model1 = gradiente_descendiente(train['Oldpeak'], train['HeartDisease'], 10000, [0,0], 0.001)

theta final 1 =>  -0.6800492768937707
theta final 2 =>  1.0290949998346437


### Modelo 2 (prueba con variable "Age")

In [ ]:
theta_model2 = gradiente_descendiente(train['Age'], train['HeartDisease'], 10000, [.1,.1], 0.001)

theta final 1 =>  -2.58170503764437
theta final 2 =>  0.2213861625528999


### Modelo 3 (prueba con variable "FastingBS")

In [ ]:
theta_model3 = gradiente_descendiente(train['FastingBS'], train['HeartDisease'], 10000, [0,0], 0.001)

theta final 1 =>  -0.1135828854804322
theta final 2 =>  1.3192435662564228


## Evaluación
Antes de poder evaluar nuestro modelo, necesitamos agrupar las predicciones de cada uno, para poder compararlas:

In [ ]:
# Inicializar acumuladores
predictions_model1 = []
predictions_model2 = []
predictions_model3 = []

# Barrer la entrada y estimar las salidas:
for i in range(len(test)):
  predictions_model1.append(round(h(train['Oldpeak'].iloc[i], theta_model1)))
  predictions_model2.append(round(h(train['Age'].iloc[i], theta_model2)))
  predictions_model3.append(round(h(train['FastingBS'].iloc[i], theta_model3)))

results_model1 = {'Predictions': predictions_model1, 'Real Value': test['HeartDisease']}
df_model1 = pd.DataFrame(results_model1)

results_model2 = {'Predictions': predictions_model2, 'Real Value': test['HeartDisease']}
df_model2 = pd.DataFrame(results_model2)

results_model3 = {'Predictions': predictions_model3, 'Real Value': test['HeartDisease']}
df_model3 = pd.DataFrame(results_model3)

#### Comparación de predicciones con subset de pruebas

In [ ]:
df_model1.head()

,Predictions,Real Value
668,0,0
30,1,1
377,0,1
535,1,1
807,1,0


In [ ]:
df_model2.head()

,Predictions,Real Value
668,1,0
30,1,1
377,1,1
535,1,1
807,1,0


In [ ]:
df_model3.head()

,Predictions,Real Value
668,0,0
30,1,1
377,1,1
535,1,1
807,1,0


In [ ]:
# Inicializar contadores para VP, FP, VN, FN (modelo 1)

#variables para matriz de confusion del modelo 1 (oldpeak)
VP_m1 = 0
FP_m1 = 0
VN_m1 = 0
FN_m1 = 0

#variables para matriz de confusion del modelo 2 (age)
VP_m2 = 0
FP_m2 = 0
VN_m2 = 0
FN_m2 = 0

#variables para matriz de confusion del modelo 2 (FastingBS)
VP_m3 = 0
FP_m3 = 0
VN_m3 = 0
FN_m3 = 0

# Barrer datos reales y predicciones (modelo 1)
for i in range(len(df_model1)):
    value = df_model1['Real Value'].iloc[i]
    prediction = df_model1['Predictions'].iloc[i]

    # Analizar opciones para dato real == 1
    if value == 1:
      # Analizar si predicción == 1 (VP) o == 0 (FN)
      if prediction == 1:
        VP_m1 += 1
      else:
        FN_m1 += 1

    # Analizar opciones para dato real == 0
    if value == 0:
      # Analizar si predicción == 1 (FP) o == 0 (VN)
      if prediction == 1:
        FP_m1 += 1
      else:
        VN_m1 += 1


# Barrer datos reales y predicciones (modelo 1)
for i in range(len(df_model2)):
    value = df_model2['Real Value'].iloc[i]
    prediction = df_model2['Predictions'].iloc[i]

    # Analizar opciones para dato real == 1
    if value == 1:
      # Analizar si predicción == 1 (VP) o == 0 (FN)
      if prediction == 1:
        VP_m2 += 1
      else:
        FN_m2 += 1

    # Analizar opciones para dato real == 0
    if value == 0:
      # Analizar si predicción == 1 (FP) o == 0 (VN)
      if prediction == 1:
        FP_m2 += 1
      else:
        VN_m2 += 1

# Barrer datos reales y predicciones (modelo 1)
for i in range(len(df_model3)):
    value = df_model3['Real Value'].iloc[i]
    prediction = df_model3['Predictions'].iloc[i]

    # Analizar opciones para dato real == 1
    if value == 1:
      # Analizar si predicción == 1 (VP) o == 0 (FN)
      if prediction == 1:
        VP_m3 += 1
      else:
        FN_m3 += 1

    # Analizar opciones para dato real == 0
    if value == 0:
      # Analizar si predicción == 1 (FP) o == 0 (VN)
      if prediction == 1:
        FP_m3 += 1
      else:
        VN_m3 += 1



print("VP Modelo 1 = ", VP_m1)
print("FP Modelo 1 = ", FP_m1)
print("VN Modelo 1 = ", VN_m1)
print("FN Modelo 1 = ", FN_m1)

print("VP Modelo 2 = ", VP_m2)
print("FP Modelo 2 = ", FP_m2)
print("VN Modelo 2 = ", VN_m2)
print("FN Modelo 2 = ", FN_m2)

print("VP Modelo 3 = ", VP_m3)
print("FP Modelo 3 = ", FP_m3)
print("VN Modelo 3 = ", VN_m3)
print("FN Modelo 3 = ", FN_m3)

VP Modelo 1 =  73
FP Modelo 1 =  61
VN Modelo 1 =  51
FN Modelo 1 =  91
VP Modelo 2 =  164
FP Modelo 2 =  112
VN Modelo 2 =  0
FN Modelo 2 =  0
VP Modelo 3 =  36
FP Modelo 3 =  17
VN Modelo 3 =  95
FN Modelo 3 =  128


#### Matrices de confusion y metricas de desempeño para el subset de entrenamiento y de pruebas

Para estimar qué tan bueno es cada clasificador, debemos generar las matrices de confusión y las correspondientes métricas:

$accuracy = \frac{VP+VN}{VP+VN+FP+FN}$
- Accuracy (Exactitud): Calcula la proporción de predicciones correctas totales del modelo en relación con el número total de predicciones. Es útil cuando las clases están balanceadas.

$precision = \frac{VP}{VP+FP}$
- Precision (Precisión): Evalúa la proporción de instancias positivas que el modelo clasifica correctamente en relación con todas las instancias que predijo como positivas. Es especialmente importante cuando los falsos positivos son costosos o problemáticos.

$recall = \frac{VP}{VP+FN}$
- Recall (Recuerdo o Sensibilidad): Mide la proporción de positivos reales que el modelo identifica correctamente en relación con todos los positivos reales presentes en los datos.


$F1 = \frac{2\cdot precision \cdot recall}{precision+recall}$
- F1 Score: Es una métrica que combina precisión y recall para proporcionar una medida equilibrada del rendimiento del modelo, especialmente cuando hay un desequilibrio entre las clases.




In [ ]:
# Calcular métricas para modelo 1
acccuracy_m1 = (VP_m1 + VN_m1) / (VP_m1 + VN_m1 + FP_m1 + FN_m1)
precision_m1 = (VP_m1) / (VP_m1 + FP_m1)
recall_m1 = (VP_m1) / (VP_m1 + FN_m1)
f1_m1 = (2 * precision_m1 * recall_m1) / (precision_m1 + recall_m1)

print("Accuracy Modelo 1 = ", acccuracy_m1)
print("Precision Modelo 1 = ", precision_m1)
print("Recall Modelo 1 = ", recall_m1)
print("F1 Modelo 1 = ", f1_m1)

print("------------------------------")

# Calcular métricas para modelo 2
acccuracy_m2 = (VP_m2 + VN_m2) / (VP_m2 + VN_m2 + FP_m2 + FN_m2)
precision_m2 = (VP_m2) / (VP_m2 + FP_m2)
recall_m2 = (VP_m2) / (VP_m2 + FN_m2)
f1_m2 = (2 * precision_m2 * recall_m2) / (precision_m2 + recall_m2)

print("Accuracy Modelo 2 = ", acccuracy_m2)
print("Precision Modelo 2 = ", precision_m2)
print("Recall Modelo 2 = ", recall_m2)
print("F1 Modelo 2 = ", f1_m2)

print("------------------------------")

# Calcular métricas para modelo 2
acccuracy_m3 = (VP_m3 + VN_m3) / (VP_m3 + VN_m3 + FP_m3 + FN_m3)
precision_m3 = (VP_m3) / (VP_m3 + FP_m3)
recall_m3 = (VP_m3) / (VP_m3 + FN_m3)
f1_m3 = (2 * precision_m3 * recall_m3) / (precision_m3 + recall_m3)

print("Accuracy Modelo 3 = ", acccuracy_m3)
print("Precision Modelo 3 = ", precision_m3)
print("Recall Modelo 3 = ", recall_m3)
print("F1 Modelo 3 = ", f1_m3)

Accuracy Modelo 1 =  0.4492753623188406
Precision Modelo 1 =  0.5447761194029851
Recall Modelo 1 =  0.4451219512195122
F1 Modelo 1 =  0.4899328859060403
------------------------------
Accuracy Modelo 2 =  0.5942028985507246
Precision Modelo 2 =  0.5942028985507246
Recall Modelo 2 =  1.0
F1 Modelo 2 =  0.7454545454545455
------------------------------
Accuracy Modelo 3 =  0.4746376811594203
Precision Modelo 3 =  0.6792452830188679
Recall Modelo 3 =  0.21951219512195122
F1 Modelo 3 =  0.3317972350230415


#### Analisis de resultados de las metricas de evaluacion
 Despues de calcular los valores de la matriz de confusión para cada una de las implementaciones del modelo y el posterior calculo de las metricas de evaluacion en base a estas se encontró lo siguiente:

 1. La variable que mejor desempeño tiene para la prediccion de clasificacion de la variable objetivo es FastingBS acorde a el analisis general de todas las metricas.
 2. Pese a que la variable Age parece tener un mejor accuracy, el analisis de las otras medidas muestra que su desempeño no es optimo.
 3. Se propone hacer un modelo donde se pueda hacer un analisis de las 3 variables en conjunto en busqueda de resultados mas balanceados.